Importing required libraries

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import PIL
from PIL import Image
from skimage.color import rgb2gray
from scipy import ndimage as ndi
import cv2
import os
from os import listdir
from sklearn.utils import shuffle

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Scanning images from directory

In [ ]:
directory_root = "/content/drive/My Drive/Colab Notebooks/Dataset"
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for image in plant_disease_folder_list:
            image_directory = f"{directory_root}/{plant_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                image_list.append(image_directory)
                label_list.append(plant_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

img_info = pd.DataFrame({'image_path':image_list,'label':label_list})
print(img_info.head())
print(len(img_info))

Labelling images with disease type

In [ ]:
#new column (empty)
img_info["labels_integer"] = None
#index of new column
index_labels_integer = img_info.columns.get_loc("labels_integer")
#index of species column
index_species = img_info.columns.get_loc("label")
#to assign numeric labels starting with 0 for the first species
k = 0 
for i in range(len(img_info)):
    if i == 0:
        img_info.iloc[i, index_labels_integer] = k #here, k == 0
    if i > 0:
        if img_info.iloc[i-1, index_species] == img_info.iloc[i, index_species]:
            img_info.iloc[i, index_labels_integer] = k
        else:
            k += 1
            img_info.iloc[i, index_labels_integer] = k
img_info.tail()


Loading and saving data

In [ ]:
img_info.image_path[0]

In [ ]:
img_info = shuffle(img_info)
list_vectors = []

for image_path in img_info.image_path:
    #read as rgb array
    img = Image.open(image_path)
    type(img)
    size = (64, 64)
    img = img.resize(size, PIL.Image.ANTIALIAS)  
    img_array = np.array(img)
    #append image vector to list
    list_vectors.append(img_array)
    
print(len(list_vectors))

X = np.stack((list_vectors))
Y =  img_info['labels_integer']
print(Y.shape)

X = X/255
Y_one_hot = keras.utils.to_categorical(Y, num_classes=3)
print(Y.shape, Y_one_hot.shape)

np.savez("x_images_arrayscnn", X)
np.savez("y_numeric_labelscnn", Y_one_hot)

x_npz = np.load("x_images_arrayscnn.npz")
X = x_npz['arr_0']

y_npz = np.load("y_numeric_labelscnn.npz")
Y_one_hot = y_npz['arr_0']

print(X.shape)

In [ ]:
#Splitting the data to test train and validation data
split_train = 0.8 #train 0.8, validate 0.1, test 0.1
split_val = 0.9
index_train = int(split_train*len(X))
index_val = int(split_val*len(X))

X_train = X[:index_train]
X_val = X[index_train:index_val]
X_test = X[index_val:]

Y_train = Y_one_hot[:index_train]
Y_val = Y_one_hot[index_train:index_val]
Y_test = Y_one_hot[index_val:]

print(X_train.shape)

In [ ]:
print("Training data shape",X_train.shape,"\nTest data shape",X_test.shape)
print("\nTraining label shape",Y_train.shape,"\nTest label shape",Y_test.shape)

In [ ]:
#converting the y_train to labels
label =[]
for i in range(Y_train.shape[0]):
  label.append(Y_train[i].argmax())


In [ ]:
data = pd.DataFrame(list(zip(X_train,label)),columns = ['features','label'])
data = data.replace({'label': {0: 'Brown Spot', 1: 'Leaf Smut', 2: 'Bacterial Leaf Blight'}})

In [ ]:
data

In [ ]:
label

In [ ]:
ax = data['label'].value_counts().plot(kind='barh', figsize=(10,7),
                                        color="coral", fontsize=13);
ax.set_alpha(0.8)
ax.set_title("Label Counts", fontsize=18)
# #ax.set_xticks([0, 5, 10, 15, 20])
ax.set_xticks([])
# create a list to collect the plt.patches data
totals = []

# find the values and append to list
for i in ax.patches:
    totals.append(i.get_width())

# set individual bar lables using above list
total = sum(totals)
label=[47,31,31]
# set individual bar lables using above list
for i in ax.patches:
    # get_width pulls left or right; get_y pushes up or down
    ax.text(i.get_width()+.3, i.get_y()+.38, \
            str(round((i.get_width())*1, 2)), fontsize=15,color='dimgrey')

# invert for largest on top 
ax.invert_yaxis()

In [ ]:
data.label.value_counts()

In [ ]:
#CNN model
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3]) 
num_classes = 3

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.7))
model.add(Flatten())
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=200, batch_size=64, validation_data=(X_val, Y_val))


In [ ]:
model.summary()

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
plt.figure(figsize=(8,8))
plt.subplots_adjust(0.6, 0.5, 2, 1.7)
disease = ['Bacterial Leaf Blight','Brown Spot','Leaf Smut']
print("Observing images")
for i in range (0,8):
  plt.subplot(3,3,i+1)
  result = model.predict(X_train[i].reshape(1,64,64,3))
  plt.imshow(X_test[i])
  plt.title("Given Label:{} \n Predicted Label: {}".format(disease[Y_test[i].argmax()],disease[result.argmax()]))
  #\n Predicted Label: {},disease[result.argmax()]
  plt.xticks([])
  plt.yticks([])
  #plt.tight_layout()
  #plt.show()


In [ ]:
#Saving the model
import pickle
from multiprocessing import Queue
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
#evaluating the model
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.evaluate(X_test, Y_test)
loaded_model.predict(X_test[13].reshape(1,64,64,3))
print(result)

# New Section

In [ ]:
#loading model
from sklearn.externals import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(model, 'disease_detection.h5')